# Embeddings and Sentence Classification

<center>
    <img src="./assets/embeddings.jpeg">
</center>

### Introduction

Embeddings are a way to represent words (or more generally, *tokens*) as vectors. These vectors are useful for many tasks in NLP, including but not limited to: Text Generation, Machine Translation, and Sentence Classification. In this notebook, we will be exploring the concept of Embeddings, and using them for Sentence Classification.

After this notebook, you should be able to:

- Understand the concepts of Embeddings and Vector Similarity.

- Use pre-trained Embeddings for Sentence Classification.

### Instructions

- Follow along with the notebook, filling out the necessary code where instructed.

- <span style="color: red;">Read the Submission Instructions and Plagiarism Policy in the attached PDF.</span>

- <span style="color: red;">Make sure to run all cells for credit.</span>

- <span style="color: red;">Do not remove any pre-written code.</span> We will be using the `print` statements to grade your assignment.

- <span style="color: red;">You must attempt all parts.</span> Do not assume that because something is for 0 marks, you can leave it - it will definitely be used in later parts.

In [1]:
import re

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split

import gensim.downloader as api
from gensim.models.word2vec import Word2Vec

from gpt4all import Embed4All

import torch
import torch.nn as nn

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Exploring Embeddings

Put simply, Embeddings are fixed-size **dense** vector representations of tokens in natural language. This means you can represent words as vectors, sentences as vectors, even other entities like entire graphs as vectors.

So what really makes them different from something like One-Hot vectors?

What's special is that they have semantic meaning baked into them. This means you can model relationships between entities in text, which itself leads to a lot of fun applications. All modern architectures make use of Embeddings in some way.

You can read more about them [here](https://aman.ai/primers/ai/word-vectors/).

We will be using *pretrained* Embeddings: this means that we will be using Embeddings that have already been trained on a large corpus of text. This is because training Embeddings from scratch is a very computationally expensive task, and we don't have the resources to do so. Fortunately, there were some good samaritans who have already done this for us, and we can use their publicly available Embeddings for our own tasks.


This part will allow you to explore what Embeddings are. We will load in pretrained Embeddings here and examine some of their properties. If you're interested, feel free look up the [Word2Vec model](https://arxiv.org/abs/1301.3781): this is the model that was trained to give us the embeddings you will see below.

In [2]:
# Download the pretrained word2vec model (this may take a few minutes)
corpus = api.load('text8')
w2vmodel = Word2Vec(corpus)

print("Done loading word2vec model!")

Done loading word2vec model!


Now that we've loaded in the Embeddings, we can create an Embedding **layer** in PyTorch, `nn.Embedding`, that will perform the processing step for us.

Note in the following cell how there is a given **vocab size** and **embedding dimension** for the model: this is important to note because some sets of Embeddings may be defined for a large set of words (a large vocab), whereas older ones perhaps have a smaller set (a small vocab); the Embedding dimension essentially tells us how many *features* have been learned for a given word, that will allow us to perform further processing on top of.

In [3]:
# Define embedding layer using gensim
embedding_layer = nn.Embedding.from_pretrained(torch.FloatTensor(w2vmodel.wv.vectors))

# Get some information from the w2vmodel
print(f"Vocab size: {len(w2vmodel.wv.key_to_index)}")

print(f"Some of the words in the vocabulary:\n{list(w2vmodel.wv.key_to_index.keys())[:10]}")

print(f"Embedding dimension: {w2vmodel.wv.vectors.shape[1]}")

Vocab size: 71290
Some of the words in the vocabulary:
['the', 'of', 'and', 'one', 'in', 'a', 'to', 'zero', 'nine', 'two']
Embedding dimension: 100


Now, for a demonstration, we instantiate two words, turn them into numbers (encoding them via their index in the vocab), and pass them through the Embedding layer. 

Note how the resultant Embeddings both have the same shape: 1 word, and 100 elements in the vector.

In [4]:
# Take two words and get their embeddings
word1 = "king"
word2 = "queen"

def word2vec(word):
    return embedding_layer(torch.LongTensor([w2vmodel.wv.key_to_index[word]]))

king_embedding = word2vec(word1)
queen_embedding = word2vec(word2)

print(f"Embedding Shape for '{word1}': {king_embedding.shape}")
print(f"Embedding Shape for '{word2}': {queen_embedding.shape}")

Embedding Shape for 'king': torch.Size([1, 100])
Embedding Shape for 'queen': torch.Size([1, 100])


When we have vectors whose scale is arbitrary, one nice way to measure how *similar* they are is with the Cosine Similarity measure.


$$ \text{Cosine Similarity}(\mathbf{u},\mathbf{v}) = \frac{\mathbf{u} \cdot \mathbf{v}}{\|\mathbf{u}\| \|\mathbf{v}\|} $$


We can apply this idea to our Embeddings. To see how "similar" two words are to the model, we can generate their Embeddings and take the Cosine Similarity of them. This will be a number between -1 and 1 (just like the range of the cosine function). When the number is close to 0, the words are not similar.

In [5]:
def cosine_similarity(vec1, vec2):
    '''
    Computes the cosine similarity between two vectors
    '''

    # TODO: Compute the cosine similarity between the two vectors (using PyTorch)
    # Step1: Get the dot product of the two vectors
    # Step2: Calculate the Norm of the First Vector
    # Step3: Calculate the Norm of the Second Vector
    # Step4: Calulcate the Product of the Two Norms
    # Step5: Divide the Dot Product of the Two Vectors with the Product of the Two Norms
    
    dot_product = torch.dot(vec1, vec2)
    # print("dot_product:",dot_product)
    norm_vec1 = torch.norm(vec1)
    # print("norm_vec1",norm_vec1)
    norm_vec2 = torch.norm(vec2)
    # print("norm_vec2",norm_vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    # print("similarity",similarity)
    return similarity.item()

def compute_word_similarity(word1, word2):
    '''
    Takes in two words, computes their embeddings and returns the cosine similarity
    '''
    # Step1: Making Embedding using the word2vec function for the 2 words
    # Step2: Squeeze the Word Embedding to bring them to a 1D Tensor
    # Step3: Get the cosine similarity using the two squeezed embedding vectors recieved in the previous step, by passing those vectors to the cosine similarity function
    # Step4: Return the Answer back
    

    embedding1 = word2vec(word1)
    embedding2 = word2vec(word2)
    
    embedding1_squeezed=embedding1.squeeze()
    embedding2_squeezed=embedding2.squeeze()
    
    similarity = cosine_similarity(embedding1_squeezed, embedding2_squeezed)
    return similarity

    # Own Function
def is_word_in_embeddings(word, embedding_model):
    '''
    Checks if a word is present in the vocabulary of pre-trained embeddings.
    Returns True if the word is found, False otherwise.
    '''
    if word in embedding_model.wv.key_to_index:
        return True
    else:
        return False
    
# TODO: Define three words (one pair should be similar and one pair should be dissimilar) and compute their similarity
word1 = "king"
word2 = "queen"
word3 = "water"
print(f"Similarity between '{word1}' and '{word2}': {compute_word_similarity(word1, word2)}")
print(f"Similarity between '{word1}' and '{word3}': {compute_word_similarity(word1, word3)}")

Similarity between 'king' and 'queen': 0.6943550705909729
Similarity between 'king' and 'water': -0.015577456913888454


In [6]:
# Run this cell if you're done with the above section
del embedding_layer

## Sentence Classification Classification with Sentence Embeddings [20 Marks]

Now let's move on to an actual application: classifying whether a tweet is about a real disaster or not. As you can imagine, this could be a valuable model when monitoring social media for disaster relief efforts.

Since we are using Sentence Embeddings, we want something that will take in a sequence of words and throw out a single fixed-size vector. For this task, we will make use of an LLM via the `gpt4all` library.

This library will allow us to generate pretrained embeddings for sentences, that we can use as **features** to feed to any classifier of our choice.

In [7]:
# Read in the data here
df = pd.read_csv("./disaster_tweets.csv")
df = df[["text", "target"]]

# Split the data
train, val = train_test_split(df, test_size=0.2, random_state=42)
print(train.shape, val.shape)

(6090, 2) (1523, 2)


In [8]:
print(train)

                                                   text  target
4996  Courageous and honest analysis of need to use ...       1
3263  @ZachZaidman @670TheScore wld b a shame if tha...       0
4907  Tell @BarackObama to rescind medals of 'honor'...       1
2855  Worried about how the CA drought might affect ...       1
4716  @YoungHeroesID Lava Blast &amp; Power Red #Pan...       0
...                                                 ...     ...
5226  @Eganator2000 There aren't many Obliteration s...       0
5390  just had a panic attack bc I don't have enough...       0
860   Omron HEM-712C Automatic Blood Pressure Monito...       0
7603  Officials say a quarantine is in place at an A...       1
7270  I moved to England five years ago today. What ...       1

[6090 rows x 2 columns]


Before jumping straight to Embeddings, since our data is sourced from the cesspool that is Twitter, we should probably do some cleaning. This can involve the removal of URLs, punctuation, numbers that don't provide any meaning, stopwords, and so on.'

In the following cell, write functions to clean the sentences. You are allowed to add more functions if you wish, but the ones provided are the bare minimum.

**Note:** After cleaning your sentences, it is possible that you may end up with empty sentences (or some that are so short they have lost all meaning). In this event, since we want to demonstrate setting up a Sentence Classification task, you should remove them from your dataset (data cleaning is not the center of this notebook).

In [9]:
# TODO: Clean the sentences (5 marks)


# TODO: Fill out the following functions, adding more if desired

def lowercase(txt):

    return txt.lower()

def remove_punctuation(txt):
    punctuation_pattern=r'[^\w\s]'
    txt_without_punctuation=re.sub(punctuation_pattern, '', txt)
    return txt_without_punctuation

def remove_stopwords(txt):
    stopwords_pattern=r'[^\w\s]'
    txt_without_stopwords=re.sub(stopwords_pattern, '', txt)
    return txt_without_stopwords

def remove_numbers(txt):
    number_pattern = r'\d+'
    text_without_numbers = re.sub(number_pattern, '', txt)
    return text_without_numbers

def remove_url(txt):
    url_pattern=r'http\S+'
    text_without_url = re.sub(url_pattern, '', txt)
    return text_without_url

def normalize_sentence(txt):
    '''
    Aggregates all the above functions to normalize/clean a sentence
    '''
    txt = lowercase(txt)
    txt = remove_punctuation(txt)
    txt = remove_stopwords(txt)
    txt = remove_numbers(txt)
    txt = remove_url(txt)
    return txt

# TODO: Clean the sentences
train['clean_text'] = train['text'].apply(normalize_sentence)
val['clean_text'] = val['text'].apply(normalize_sentence)
# df['clean_text'] = df['text'].apply(normalize_sentence)

# TODO: Filter sentences that are too short (less than 20ish characters)
train = train[train['clean_text'].apply(lambda x: len(x) > 20)]
val = val[val['clean_text'].apply(lambda x: len(x) > 20)]
# df = df[df['clean_text'].apply(lambda x: len(x) > 20)]


# Check the shapes of cleaned train and validation data
print(train.shape, val.shape)

(5966, 3) (1501, 3)


Now for the fun part, creating our Embeddings!

We will be using the `gpt4all.Embed4All` class for this purpose. You can look up the documentation [here](https://docs.gpt4all.io/gpt4all_python_embedding.html#gpt4all.gpt4all.Embed4All.embed).

This functionality makes use of a model called [Sentence-BERT](https://arxiv.org/abs/1908.10084). This is a Transformer-based model that has been trained on a large corpus of text, and is able to generate high-quality Sentence Embeddings for us.

In [10]:
# TODO: Generate embeddings for train and validation sentences (5 marks)

feature_extractor = Embed4All()

# TODO: Encode the train samples


# TODO: Encode the train sentences
# train_sentences = train['clean_text'].tolist()
# train_embeddings = feature_extractor.embed(train_sentences)
train_sentences = train['clean_text'].tolist()
train_embeddings = []

for sentence in train_sentences:
    embedding = feature_extractor.embed(sentence)
    train_embeddings.append(embedding)

# TODO: Encode the validation sentences
# val_sentences = val['clean_text'].tolist()
# val_embeddings = feature_extractor.embed(val_sentences)
val_sentences = val['clean_text'].tolist()
val_embeddings = []

for sentence in val_sentences:
    embedding = feature_extractor.embed(sentence)
    val_embeddings.append(embedding)


# TODO: Ready the labels
train_labels = train['target'].tolist()
val_labels = val['target'].tolist()

Now with our Embeddings ready, we can move on to the actual classification task.

You have the choice of using **any** classifier you wish. You can use a simple Logistic Regression model, get fancy with Support Vector Machines, or even use a Neural Network. The choice is yours.

We will be looking for a model with a **Validation Accuracy** of around $0.8$. You must also use this model to make predictions on your own provided inputs, after completing the `predict` function.

In [11]:
# TODO: Get 0.8 Validation Acc with a Classifier (5 marks)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Convert embeddings to numpy arrays
train_embeddings = np.array(train_embeddings)
val_embeddings = np.array(val_embeddings)

print(train_embeddings.shape)

# Reshape the embeddings for Logistic Regression
train_embeddings = [emb.reshape(-1) for emb in train_embeddings]
val_embeddings = [emb.reshape(-1) for emb in val_embeddings]

# Create a Logistic Regression model
model = LogisticRegression(max_iter=1000, random_state=42)

# Fit the model
model.fit(train_embeddings, train_labels)

# Predict on the validation set
val_predictions = model.predict(val_embeddings)

# Calculate validation accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Validation Accuracy: {accuracy:.4f}")


(5966, 384)
Validation Accuracy: 0.8081


In [12]:
def predict(sentence, clf):
    '''
    Takes in a sentence and returns the predicted class along with the probability
    '''
    # TODO: Clean and encode the sentence
    cleaned_sentence = normalize_sentence(sentence)
    
    sentence_embedding = feature_extractor.embed(cleaned_sentence)
    
    sentence_embedding = np.array(sentence_embedding)
    
    sentence_embedding = sentence_embedding.reshape(1, -1)
    
    # TODO: Predict the class and probability
    predicted_class = clf.predict(sentence_embedding)[0]  
    probability = clf.predict_proba(sentence_embedding)[0][predicted_class]
    
    return predicted_class, probability

# Example Sentences
sentences_to_predict = [
    "What are natural disasters.",
    "I love sunny days at the Cliffton Beach.",
    "The strongest earthquake in Pakistan happened on 09/24/2013"
]

# Predict on example sentences
for sentence in sentences_to_predict:
    pred_class, prob = predict(sentence, model)
    print(f"Sentence: {sentence}")
    print(f"Predicted Class: {pred_class}, Probability: {prob:.4f}\n")

Sentence: What are natural disasters.
Predicted Class: 1, Probability: 0.8829

Sentence: I love sunny days at the Cliffton Beach.
Predicted Class: 0, Probability: 0.6642

Sentence: The strongest earthquake in Pakistan happened on 09/24/2013
Predicted Class: 1, Probability: 0.8867



Hopefully now you realize the power of Embeddings, and the usefulness of pretrained models.

# Fin.